In [ ]:
# from google.colab import drive # 挂载谷歌云盘
# drive.mount('/content/drive')
# !nvidia-smi # 显示显卡信息
# ''' 符号%代表一直生效，！代表执行完立马结束，不会生效，所以进入目录用% '''
# %cd /content/drive/MyDrive/timeSerise
# ''' 支持的 常用命令1.ls  2.wget  3.gdoint(int(int(int(w))))n  4.mkdir  5.pwd '''
# !ls
# !pip install pyti
# !pip install akshare

In [4]:
from data_provider.data_creat import *
import akshare as ak
from datetime import datetime

In [107]:
class Args:
    '''数据获取'''
    fuquan = 'hfq'# 设置复权方式,adjust=空选择的不复权，qfq是前复权，应该用hfq后复权来进行量化分析
    period = 'daily' # 拉取时间周期{'daily', 'weekly', 'monthly'}
    start_date = '20151201'  # 20151201   20221021  下载数据的开始日期,1就是公司上市时间
    end_date = '20231220'  # 下载数据的结束日期,如果1则到最后一天,如果-1是昨天.
    start_date_time = pd.to_datetime('2015-12-01')
    end_date_time = pd.to_datetime('2023-12-20')
    label_n = 5 # 预测未来连续多少天的收益率
    root_path = './dataset/'
    air_data = 'shanghai-air-quality.csv' # 空气质量数据
    # air_key = "f75cec5c59f7abf51b27ab6433ba4d7637fa2153"
    national_debt = '中国十年期国债收益率历史数据.csv' # 国债数据
    USD_exchange_rate = 'USD_CNH历史数据.csv' # 美元人民币汇率
    HK_HSI = '香港恒生指数历史数据.csv'
    
    # 预测目标y是否替换成1或者0
    zhangfu = 0.10  # 预测涨幅大于等于3%的为1，小于3%的为0
    label_ch = True  # 如果是True ，预测n天以后上涨大于变量zhangfu为1，小于为0

    # 是否合并全部股票数据
    all = False
    data_addzero = 60 # 当all = True时，用于训练的数据集分割，前面补0的长度
    
    # 数据修剪
    start = 103 # 删除前24行（start=25），因为macd算不出来
    end = 0 # (end = label_n if all else 0)删除最后部分需要预测天数label_n的数据，算出来是0.如果单只股票预测，那么不需要删除
    final_data_feat =  ['index', 'Volume','Tom_Chg'] # 删除不需要列的标签

# 创建参数对象
args = Args()
dataset_lists = []

In [108]:
# 获取上海空气数据
if args.air_data:
    print("空气数据数据读取中")
    # https://aqicn.org/historical/#city:shanghai
    air_df = pd.read_csv(args.root_path + args.air_data)
    air_df.columns = ['date', 'air_pm25', 'air_pm10', 'air_o3', 'air_no2', 'air_so2', 'air_co']
    # 转换为时间格式、设置时间为索引、去除时区信息
    air_df['date'] = pd.to_datetime(air_df['date'], errors='coerce')
    air_df = air_df.set_index('date')
    air_df = air_df.tz_localize(None)
    # 只保留air_pm25列
    air_df = air_df[['air_pm25']]
    # 截取时间范围的数据
    filtered_air_df = air_df[(air_df.index >= args.start_date_time) & (air_df.index <= args.end_date_time)]
    dataset_lists.append(filtered_air_df)

空气数据数据读取中


In [109]:
# 获取国债数据
if args.national_debt:
    print("国债数据读取中")
    # https://cn.investing.com/rates-bonds/china-10-year-bond-yield-historical-data?dt_dapp=1
    national_debt_df = pd.read_csv(args.root_path + args.national_debt)
    # 重命名列名
    national_debt_df.columns = ['date', 'national_close', 'national_open', 'national_high', 'national_low', 'national_change']
    national_debt_df['date'] = pd.to_datetime(national_debt_df['date'], errors='coerce')
    national_debt_df = national_debt_df.set_index('date')
    national_debt_df = national_debt_df.tz_localize(None)
    # 只保留'national_close'列
    national_debt_df = national_debt_df[['national_close']]
    filtered_national_debt_df = national_debt_df[(national_debt_df.index >= args.start_date_time) & (national_debt_df.index <= args.end_date_time)]
    dataset_lists.append(filtered_national_debt_df)

国债数据读取中


In [110]:
# 获取人民币汇率
if args.USD_exchange_rate:
    print("人民币汇率数据读取中")
    # https://cn.investing.com/currencies/usd-cnh-historical-data
    USD_exchange_rate_df = pd.read_csv(args.root_path + args.USD_exchange_rate)
    USD_exchange_rate_df.columns = ['date', 'USD_close', 'USD_open', 'USD_high', 'USD_low', 'USD_volume', 'USD_change']
    USD_exchange_rate_df['date'] = pd.to_datetime(USD_exchange_rate_df['date'], errors='coerce')
    USD_exchange_rate_df = USD_exchange_rate_df.set_index('date')
    USD_exchange_rate_df = USD_exchange_rate_df.tz_localize(None)
    # 只保留'USD_close'列
    USD_exchange_rate_df = USD_exchange_rate_df[['USD_close']]
    filtered_USD_exchange_rate_df = USD_exchange_rate_df[(USD_exchange_rate_df.index >= args.start_date_time) & (USD_exchange_rate_df.index <= args.end_date_time)]
    dataset_lists.append(filtered_USD_exchange_rate_df)

人民币汇率数据读取中


In [111]:
# 获取香港恒生指数
if args.HK_HSI:
    print("香港恒生指数数据读取中")
    # https://cn.investing.com/indices/hang-sen-40-historical-data
    HK_HSI_df = pd.read_csv(args.root_path + args.HK_HSI)
    HK_HSI_df.columns = ['date', 'HK_HSI_close', 'HK_HSI_open', 'HK_HSI_high', 'HK_HSI_low', 'HK_HSI_volume', 'HK_HSI_change']
    HK_HSI_df['date'] = pd.to_datetime(HK_HSI_df['date'], errors='coerce')
    HK_HSI_df = HK_HSI_df.set_index('date')
    HK_HSI_df = HK_HSI_df.tz_localize(None)
    # 只保留'HK_HSI_close'列
    HK_HSI_df = HK_HSI_df[['HK_HSI_close']]
    # 转换所有列为数值，移除逗号
    for column in HK_HSI_df.columns:
        HK_HSI_df[column] = HK_HSI_df[column].str.replace(',', '').astype(float)/1000
    # 过滤数据
    filtered_HK_HSI_df = HK_HSI_df[(HK_HSI_df.index >= args.start_date_time) & (HK_HSI_df.index <= args.end_date_time)]
    dataset_lists.append(filtered_HK_HSI_df)

香港恒生指数数据读取中


In [118]:
print(dataset_lists[3])

            HK_HSI_close
date                    
2023-12-20      16.61381
2023-12-19      16.50500
2023-12-18      16.62923
2023-12-15      16.79219
2023-12-14      16.40219
...                  ...
2015-12-07      22.20322
2015-12-04      22.23589
2015-12-03      22.41701
2015-12-02      22.47969
2015-12-01      22.38135

[1988 rows x 1 columns]


In [ ]:
# 创建股票数据列表
stock_down = ak.stock_cy_a_spot_em() # 创业板实时数据
stock_list = stock_down[~stock_down['名称'].str.contains("退|ST") & (stock_down['流通市值'] <= 1e11) & (stock_down['总市值'] >= 45e8)] # 去除退市和ST股票
file_name_cy = 'Stock_list_cy.csv'# 保存数据，编码格式为utf-8
stock_list.to_csv(args.root_path + file_name_cy,index=False,encoding='utf-8-sig')

# 读取股票列表
stock_list = pd.read_csv(args.root_path + file_name_cy) # 读取股票列表
# 将股票代码的数字转换为字符串列表
stock_list = [str(code) for code in stock_list['代码'].tolist()]

stock_list = ['600028'] # 自定义股票列表

In [8]:
from sklearn.preprocessing import StandardScaler

all_data_raw = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储原始数据
all_data_scaled = pd.DataFrame()  # 初始化一个空的 DataFrame 用于存储标准化后的数据
processed_count = 0  # 初始化计数器
total_count = len(stock_list)  # 获取总股票数量
scaler = StandardScaler()

for i in stock_list[:]:
    NUM = i
    try:
        # 下载原始数据
        raw_data = download_data(NUM, args)
        # 更新已处理股票数量计数器
        processed_count += 1
        # 检查数据长度，如果小于300则跳过此次循环
        if raw_data.shape[0] < 280:
            print(f"股票代码 {NUM} 的数据长度小于300,跳过此次循环。")
            continue
        
        # 拼接数据，添加各种参数
        ad_data = add_data(raw_data, args)
        # 添加预测标签
        # ad_data = add_label(ad_data, args)
        # 删除无效数据
        final_data_raw = sub_data(ad_data.copy(), args)
        final_data_scaled = final_data_raw.copy()
        
        # 未处理数据的前20行替换为0
        if args.data_addzero and not all_data_raw.empty:
            final_data_raw = add_zeros_to_data(final_data_raw, num_rows=args.data_addzero)

        # 标准化处理
        non_time_columns = final_data_scaled.columns[1:-1]  # 假设时间列是第一列
        final_data_scaled[non_time_columns] = scaler.fit_transform(final_data_scaled[non_time_columns])

        # 标准化处理的数据的前20行替换为0
        if args.data_addzero and not all_data_scaled.empty:
            final_data_scaled = add_zeros_to_data(final_data_scaled, num_rows=args.data_addzero)
        
        # 如果 all 为 False, 则为每个股票单独保存数据
        if not args.all:
            file_name_raw_individual = f"raw_stock_cy_{NUM}.csv"
            file_name_scaled_individual = f"scaled_stock_cy_{NUM}.csv"
            final_data_raw.to_csv(args.root_path + file_name_raw_individual, index=False)
            final_data_scaled.to_csv(args.root_path + file_name_scaled_individual, index=False)
            print(f"股票代码 {NUM} 的数据保存完毕，形状: {final_data_raw.shape}")
        else:
            # 拼接未处理的数据
            all_data_raw = pd.concat([all_data_raw, final_data_raw], ignore_index=True)
            # 拼接经过标准化的数据
            all_data_scaled = pd.concat([all_data_scaled, final_data_scaled], ignore_index=True)
            print(f"当前all_data的形状: {all_data_scaled.shape}")

    except Exception as e:
        print(f"处理股票代码 {NUM} 时出现错误: {e}")
        continue

    # 计算并打印处理进度
    progress = processed_count / total_count
    print(f"处理进度: {processed_count}/{total_count} ({progress:.2%})")

# 如果 all 为 True, 则保存合并后的数据
if args.all:
    time = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    file_name_raw = "stock_cy_all_raw.csv"
    file_name_scaled = "stock_cy_all_scaled.csv"
    all_data_raw.to_csv(args.root_path + time + file_name_raw, index=False)
    all_data_scaled.to_csv(args.root_path + time + file_name_scaled, index=False)
    print("合并数据保存完毕。")
else:
    print("单独数据保存完毕。")

获取数据时间为： 20151201 - 20231220
原始数据形状： (1961, 9)
添加数据以后形状： (1961, 33)
删除指定行、列后数据形状:  (1858, 32)
股票代码 600028 的数据保存完毕，形状: (1858, 32)
处理进度: 1/1 (100.00%)
单独数据保存完毕。
